In [1]:
import cv2
import numpy as np
import itertools
from matplotlib import pyplot as plt

# Find convolution of 2 shape

In [2]:
def getMask(mask_path):
    # path to mask image
    mask = cv2.imread(mask_path,0)
    mask_position=set()
    for row, value in enumerate(mask):
        for column, element in enumerate(value):
            if element == 255:
                mask_position.add((column,row))
    return mask_position

def getBbox(bbox):
    x,y,x_plus_w,y_plus_h = bbox 
    vehicle_position = itertools.product(range(x,x_plus_w+1),range(y,y_plus_h+1))
    return set(vehicle_position)

def getProbability2Shape(vehicle, mask):
    match = len(vehicle)-len(vehicle.difference(mask))
    bbox  = len(vehicle)
    return (match*100)/bbox

'''
Just a mark for myself
'''
def get_output_layers(net):
    '''
    get all output layer names: with yolov3 is yolo_82, 94 and 106
    ''' 
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers


def draw_prediction(img, class_id, confidences, x, y, x_plus_w, y_plus_h):
    '''
    draw a bounding box around object
    '''
    label = str(classes[class_id])

    color = COLORS[class_id]
    
#     print(x,y,x_plus_w,y_plus_h)
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

def bounding_pic(net, image, scale, size,mask):
    '''
    net: the model reading by open-cv (maybe YoLo or SSD)
    image: image need to bouding box
    scale: image pixel multiply with this
    size: a tuple contain size of image
    '''
    Width = image.shape[1]
    Height = image.shape[0]
    # Resize picture to 416x416, because YOLO take in 416x416
    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
    # set input is resized picture
    net.setInput(blob)
    # last layer of Yolo model
    outs = net.forward(get_output_layers(net))

    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.6
    # maybe our model will detect many bouding box for an object, this threshold help us take the box with equal 
    #            or higher propability
    nms_threshold = 0.4

    '''
    out is a 2D tensor like (number_of_objects, score_of_each_classes), with first five element in each row is special, 
    take e.g: out[0] = temp:
        + temp[0]: x_center of that object
        + temp[1]: y_center of that object
        + temp[2]: width of that object
        + temp[3]: height of that object
        + temp[4]: unknow value
        + from 5 to above is the score of that object to each classes => COCO have 80 class so each row contain 85 element,
            will be 15 with CIFAR,and 1005 with IMAGENET  
    '''
    for out in outs:
        for detection in out:
            scores = detection[5:]
            # get the highest score to determine its label
            class_id = np.argmax(scores)
            if class_id not in [0,1,2,3,7]:
                continue
            else:
                # score of that object, make sure more than 50% correct label
                confidence = scores[class_id]
                if confidence > 0.5:
                    # scale again with w and h
                    center_x = int(detection[0] * Width)
                    center_y = int(detection[1] * Height)
                    w = int(detection[2] * Width)
                    h = int(detection[3] * Height)
                    # remember it return x_center and y_center, not x,y, so we need to find x,y
                    x = center_x - w / 2
                    y = center_y - h / 2
                    class_ids.append(class_id)
                    confidences.append(float(confidence))
                    boxes.append([x, y, w, h])

    # detect bouding box around objects
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    # set the counting line
    lineThickness = 2
    
    for i in indices:
        i = i[0]
        box = boxes[i]
        x = round(box[0])
        y = round(box[1])
        w = round(box[2])
        h = round(box[3])
        draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
        bbox = getBbox((x,y,x+w,y+h))
        conv = getProbability2Shape(bbox,mask)
        text1 = 'Convolution: ' + str(round(conv,2)) + '%'
        text2 = 'True' if conv > 80 else 'False'
        cv2.putText(image, text1, (x, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 5)
        cv2.putText(image, 'Fault: '+text2, (x, y+30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 5)
    return image

In [64]:
config = 'YOLOv3-416\\yolov3.cfg'
name = 'coco.names'
weight = 'YOLOv3-416\\yolov3.weights'
mask_path = './cross line/mask.jpg'

classes = None

with open(name, 'r') as f:
    # generate all classes of COCO, bicycle ind = 1, car ind = 2 and motorbike ind = 3
    classes = [line.strip() for line in f.readlines()]

COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# Read the model
net = cv2.dnn.readNet(weight, config)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL)
# net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
# take shape of image in order to scale it to 416x416, first layer of Yolo CNN

scale = 0.00392

cap = cv2.VideoCapture('Z:\\Vehicle speed measuring\\video_speed measure\\front.avi')
'''
If output is not None
'''
fourcc = cv2.VideoWriter_fourcc(*'XVID')
outputVideo = cv2.VideoWriter('C:/Users/ADMINS/Desktop/areaResult.avi', fourcc, 20, (1920,1080))

#close

frameNo = 0
count = 0


# for session_0_center
mask = getMask(mask_path)


while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    thresh = 1
    frameNo += 1

    if count%thresh==0:
        # Our operations on the frame come here
        img = bounding_pic(net, frame, scale, (416,416),mask)
        #510,496 1200,471 1527,616 706,749
        img = cv2.line(img, (510,496), (1200,471), (255,0,0), 3)
        img = cv2.line(img, (1200,471), (1527,616), (255,0,0), 3)
        img = cv2.line(img, (1527,616), (706,749), (255,0,0), 3)
        img = cv2.line(img, (706,749), (510,496), (255,0,0), 3)
        cv2.namedWindow('image',cv2.WINDOW_NORMAL)
        cv2.resizeWindow('image', 800,600)
        cv2.imshow('image',img)
        outputVideo.write(frame)
    
    count += 1
    if count == thresh*30:
        count = 0
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
outputVideo.release()
cv2.destroyAllWindows()

# Find the position of vehicle

In [26]:
import numpy as np
def distanceFromPoint2Line(p, line):
    a,b,c = line[0],line[1],line[2]
    try:
        # if list or np.array shape 1
        x,y = p[0],p[1]
    except:
        # if np.array shape 2
        x,y = p[0][0], p[0][1]
    finally:
        top = abs(a*x+b*y+c)
        bottom = np.linalg.norm(np.array([a,b]))
        return top/bottom

def cosineVetorPhase(v1,v2):
    dotProduct = np.dot(v1,v2)
    normV1 = np.linalg.norm(v1)
    normV2 = np.linalg.norm(v2)
    return dotProduct/(normV1*normV2)

def checkSameSideNormalVector(p, line):
    # p is numpy array with shape [1,3] with the final element is 1
    # line is something like ax+by+c = 0, line =[a,b,c]
    return (line[0]*p[0]+line[1]*p[1]+c) > 0


# SYSTEM

In [5]:
from yolo import *
# from tracking import *

In [10]:
a = np.array([1,2])
print(np.append(a,[1]))

[1 2 1]


# vehicle_speed

In [73]:
import numpy as np
import cv2
import math

def isPointBetweenLines(p, l1, l2):
    p = np.append(np.array(p),[1])
    return np.dot(p,l1)*np.dot(p,l2)*np.dot(l1[0:2],l2[0:2]) <= 0

def getLaneForPoint(p, lines):
    for i in range(len(lines)-1):
        if isPointBetweenLines(p, lines[i], lines[i+1]):
            return i
    return None

def pointToLineDistance(p, l):
    return abs(np.dot(l,p/p[2]))/np.linalg.norm(l[0:2])

def pointToLineProjection(l, p):
    p = p/p[-1]
    c = p[0]*l[1] - p[1]*l[0]
    perpendicularLine = np.array([-l[1], l[0], c])
    intersection = np.cross(l, perpendicularLine)
    return intersection/intersection[-1]

def getFocal(vp1, vp2, pp):
    return math.sqrt(- np.dot(vp1[0:2]-pp[0:2], vp2[0:2]-pp[0:2]))


def computeCameraCalibration(_vp1, _vp2, _pp):
    vp1 = np.concatenate((_vp1, [1]))    
    vp2 = np.concatenate((_vp2, [1]))    
    pp = np.concatenate((_pp, [1]))    
    focal = getFocal(vp1, vp2, pp)
    vp1W = np.concatenate((_vp1, [focal]))    
    vp2W = np.concatenate((_vp2, [focal]))    
    ppW = np.concatenate((_pp, [0])) 
    vp3W = np.cross(vp1W-ppW, vp2W-ppW)
    vp3 = np.concatenate((vp3W[0:2]/vp3W[2]*focal + ppW[0:2], [1]))
    vp3Direction = np.concatenate((vp3[0:2], [focal]))-ppW
    roadPlane = np.concatenate((vp3Direction/np.linalg.norm(vp3Direction), [10]))
    return vp1, vp2, vp3, pp, roadPlane, focal

def getWorldCoordinagesOnRoadPlane(p, focal, roadPlane, pp):
    p = p/p[2]
    pp = pp/pp[2]
    ppW = np.concatenate((pp[0:2], [0]))
    pW = np.concatenate((p[0:2], [focal]))
    dirVec = pW - ppW
    t = -np.dot(roadPlane, np.concatenate((ppW, [1])))/np.dot(roadPlane[0:3], dirVec)
    return ppW + t*dirVec

def camera_calibration(vp1,vp2,pp):
    return computeCameraCalibration(vp1,vp2,vp3)

def calculateSpeeds(loc1, loc2, fps, scale, frame_diff, tuple_cam):
    vp1, vp2, vp3, pp, roadPlane, focal = tuple_cam
    projector = lambda p: getWorldCoordinagesOnRoadPlane(p, focal, roadPlane, pp)
    points = map(lambda p: np.array([p[0],p[1],1]), (loc1, loc2))
    points = list(points)
    
    passedDistance = scale*np.linalg.norm(points[-1]-points[-2])
    elapsedTime = abs(frame_diff)/fps
    # m/s -> km/h
    speed = passedDistance/elapsedTime * 3.6
    return speed

# vehicle

In [77]:
import numpy as np
import cv2

# from vehicle_speed import *
# from cross_red_line import *

class Vehicle:
    def __init__(self, ID, centroid, frame_appear, fps, scale, tuple_cam, bbox, 
                    allow_speed, allow_lanes, all_lanes, **kwags):
        '''
        watch ignore list in tracking
        tuple_cam include vp1,vp2, vp3, pp, roadPlane, focal (put this into main)
        calculate speed, measure lane, cross line if fault, save bbox into directionary
        delete vehicle in N time disappear
        check if thresh car problem in 1 time -> traffic jam
        # fix to have bbox in main #
        '''
        '''
        for cross_line:
            accept_line
            right_ditection = False
            area of interest
            
        '''
#         centroid_point = centroid
#         centroid_point.extend([1])
        self.ID = ID
        self.centroids = [centroid, None]
        self.frame = [frame_appear, None]
        self.bbox  = bbox # image
        self.lane = getLaneForPoint(centroid,all_lanes)
        self.catch_cross_lane = False
        self.fps = fps
        self.scale = scale
        self.speed = 0
        self.allow_lanes = allow_lanes
        self.allow_speed = allow_speed
        self.all_lanes = all_lanes
        # self.called = 0
        self.disappear = 10
        self.clock = 0
        self.stayed = False
        self.problem = False
        self.tuple_cam = tuple_cam
        self.mode = 'speed'
        self.overSpeed_path = './OverSpeed/'
        self.crossLane_path = './CrossLane/'
        self.crossRedLine_path = './crossRedLine/'
        self.problem_path = './carWithProblem/'

        if self.catch_cross_lane and (lane not in self.allow_lanes):
            self.catch_fault_vehicle(self.crossLane_path)

    def update_for_highway(self, new_bbox, new_centroid, new_frame):
        # update all element and calculate speed, instead of all the other fault
        # except cross line
        # this function update in mode 'highway' in main, can measure speed and 
        # detect the vehicle with problem

        vp1, vp2, vp3, pp, roadPlane, focal = self.tuple_cam
        laneDivLines = self.all_lanes
        self.centroids[1] = new_centroid
        self.frame[1] = new_frame
        self.bbox = new_bbox
#         if np.linalg.norm(centroids[-1]-centroids[-2]) <10:
#             # check if vehicle stand
#             self.clock += 1
# #             return None
#         if self.clock == 10:
#             # Boom
#             self.stayed = True
#             return None

        if self.catch_cross_lane and (lane not in self.allow_lanes):
            self.catch_fault_vehicle(self.crossLane_path)

        if not(getLaneForPoint(self.centroids[0], laneDivLines) is None or \
               getLaneForPoint(self.centroids[1], laneDivLines) is None):
            frame_diff = self.frame[1]-self.frame[0]
            
            if frame_diff != 5:
                print('ok')
                
            self.speed = calculateSpeeds(self.centroids[0],self.centroids[1], self.fps, 
                                self.scale, frame_diff, self.tuple_cam)
            if self.speed > self.allow_speed:
                self.catch_fault_vehicle(self.overSpeed_path)
        # self.called += 1
        self.centroids[0] = new_centroid
        self.frame[0] = new_frame
            
    # def update_for_cross_way(self, centroid, frame_appear):
        
        

    # def check_lane(self, lane):
    #     if catch_cross_lane and (lane not in self.allow_lanes):
    #         self.catch_cross_lane() 

    def catch_fault_vehicle(self, src):
        # mode: 'speed', 'other'
        ID, frame, bbox , mode = self.ID, self.frame[-1], self.bbox, self.mode
        if mode == 'speed':
            cv2.imwrite(self.overSpeed_path + str(round(self.speed,2)) + '_' + str(frame)+ "_vehicle_" + str(ID) + ".jpg", bbox)
        else:
            cv2.imwrite(str(frame)+ "_vehicle_" + str(ID) + ".jpg", bbox)


# Training

In [78]:
from sort import *
from utils import *
# from vehicle import *

from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw

import os
import utilities
import numpy as np
import cv2

def bbox2necess(image, bbox,frame,shape):
    """
    return a list, each element is a list contain [posX,posY,ID,frame_no, image bbox]
    """
    final_res=[]
    width = shape[0]
    height = shape[1]
#     print(width,height)
    for box in bbox:
#         print(box)
        x = box[0]
        y = box[1]
        w = box[2]-box[0]
        h = box[3]-box[1]
#         print(x,y,w,h)
        x_plus_w = x+w
        y_plus_h = y+h
        bbox2d = image[int(round(x)):int(round(x_plus_w)),int(round(y)):int(round(y_plus_h))]
        x_centroid = x + w/2
        y_centroid = y + h/2
        res=[x_centroid,y_centroid,(box[4]),frame,bbox2d]
        final_res.append(res)
    return final_res


def detect_video(yolo, video_type, video_path, output_path, 
                    scale, vp1, vp2, pp, allow_speed, allow_lanes,
                    all_lanes, thresh_frame):
    '''
    - Input:
        + yolo: yolo model
        + video_type: 'local' when use video in PC and 'stream' for....stream
        + video_path: path of video
        + output_path: write down if u wanna something more clearly
    - Output:
        + A tensor [x,y,ID,frame_num]
    '''
    # these thing should append into data file
    tuple_cam = computeCameraCalibration(vp1,vp2,pp)
    

    vid = cv2.VideoCapture(video_path)
    if not vid.isOpened():
        raise IOError("Couldn't open webcam or video")
    if video_type == 'stream':
        fps = vid.get(cv2.cv.CV_CAP_PROP_FPS)
    elif video_type == 'local':
        fps = vid.get(7)
        
    print('fps: {}'.format(fps))
    
    video_FourCC = cv2.VideoWriter_fourcc(*'XVID')
    video_size      = (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)),
                        int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    isOutput = True if output_path != "" else False
    if isOutput:
        print("!!! TYPE:", type(output_path), type(video_FourCC), type(fps), type(video_size))
        out = cv2.VideoWriter(output_path, video_FourCC, fps, video_size)

    tracker=Sort()
    frame_num=0
    all_vehicle={}
    ignore_set = set()
    while True:
        return_value, pic = vid.read()
        if not return_value:
            break
        image = Image.fromarray(pic)
        if frame_num%thresh_frame==0:
            if yolo.model_image_size != (None, None):
                assert yolo.model_image_size[0]%32 == 0, 'Multiples of 32 required'
                assert yolo.model_image_size[1]%32 == 0, 'Multiples of 32 required'
                boxed_image = letterbox_image(image, tuple(reversed(yolo.model_image_size)))
            else:
                new_image_size = (image.width - (image.width % 32),
                                image.height - (image.height % 32))
                boxed_image = letterbox_image(image, new_image_size)
            image_data = np.array(boxed_image, dtype='float32')
            image_data /= 255.
            image_data = np.expand_dims(image_data, 0)  # Add batch dimension.
            
            # detect for bbox right here

            out_boxes, out_scores, out_classes = yolo.sess.run(
                [yolo.boxes, yolo.scores, yolo.classes],
                feed_dict={
                    yolo.yolo_model.input: image_data,
                    yolo.input_image_shape: [image.size[1], image.size[0]],
                    K.learning_phase(): 0
                })
            final_box=[]
            label=[]
            scores=[]
            for b,lb,sc in zip(out_boxes,out_classes,out_scores):
                if lb in [2,3,5,7]:
                    final_box.append(b)
                    label.append(lb)
                    scores.append(sc)

            out_boxes=np.array(final_box)
            out_classes=np.array(label)
            out_scores=np.array(scores)
            final_boxes=np.column_stack((out_boxes,out_scores))
            final_boxes = final_boxes[np.logical_and(final_boxes[:, 4] > 0.3, final_boxes[:, 2] -
                                                        final_boxes[:, 0] < 600)]

                # Apply NMS
            indices = utilities.non_max_suppression(final_boxes, 0.9, final_boxes[:, 4])
            
            out_boxes = [final_boxes[i] for i in indices]
            out_classes= [out_classes[i] for i in indices]
            rev=(reversed(out_classes))  # for display in order since yolo reverse the list 
            out_classes=[]
            for r in rev:
                out_classes.append(r)
            out_classes=np.array(out_classes)
            bf=out_boxes
            res_track=tracker.update(np.array(out_boxes))
            # res_track return [x,y, x+w, x+y, ID]
            one_frame = bbox2necess(image = pic, bbox = res_track,frame =frame_num,
                                        shape = video_size)
            for vehicle in one_frame:
                # [posX,posY,ID,frame_no, image bbox]
#                 print(vehicle)
                ID =  int(round(vehicle[2]))
                centroid = [vehicle[0],vehicle[1]]
                frame_appear = frame_num
                bbox = vehicle[4]
                
                mode = 'speed'

                # later with clock and disappeared

                # if ID in ignore_set:
                #     continue
                if ID not in all_vehicle.keys():
                    all_vehicle[ID] = Vehicle(ID, centroid, frame_appear, fps, scale,
                                                tuple_cam, bbox, allow_speed, allow_lanes, 
                                                all_lanes, mode = mode)
                else:
                    all_vehicle[ID].update_for_highway(bbox, centroid, frame_appear)
#                     print(ID)
#                     print(all_vehicle[ID].speed)
        frame_num+=1



KalmanBoxTracker.count = 0

In [79]:
KalmanBoxTracker.count = 0
video_path = 'Z:\\Vehicle speed measuring\\video_speed measure\\front.avi'
yolo = Yolo(score = 0.3, iou = 0.45)
vp1 = [144.737, 34.7794]
vp2 = [12183.582175112755, 615.451021479187]
pp = [960.5, 540.5]
scale = 0.01822590999670784
all_lanes = [np.array([ -0.92226493,   0.38655841, 130.89136422]),
             np.array([-0.78325241,  0.62170384, 97.52101372]),
             np.array([-0.65281474,  0.7575176 , 71.40300615]), 
             np.array([  0.57,  -1.  , -47.39])]
allow_lanes = [1,2]
a = detect_video(yolo = yolo, video_type = 'local', video_path = video_path, output_path = "", scale = scale,
                vp1 = vp1, vp2 = vp2, pp = pp, allow_speed = 40, allow_lanes = allow_lanes, all_lanes = all_lanes,
                thresh_frame = 5)


YOLOv3-416/yolo.h5 model, anchors, and classes loaded.
fps: 7500.0


C:\Users\ADMINS\Anaconda3\envs\yolo\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
C:\Users\ADMINS\Anaconda3\envs\yolo\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
C:\Users\ADMINS\Anaconda3\envs\yolo\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
C:\Users\ADMINS\Anaconda3\envs\yolo\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed

C:\Users\ADMINS\Anaconda3\envs\yolo\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
C:\Users\ADMINS\Anaconda3\envs\yolo\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
C:\Users\ADMINS\Anaconda3\envs\yolo\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
C:\Users\ADMINS\Anaconda3\envs\yolo\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed

C:\Users\ADMINS\Anaconda3\envs\yolo\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
C:\Users\ADMINS\Anaconda3\envs\yolo\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
C:\Users\ADMINS\Anaconda3\envs\yolo\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
C:\Users\ADMINS\Anaconda3\envs\yolo\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed

KeyboardInterrupt: 

In [55]:
tuple_cam = computeCameraCalibration(vp1,vp2,pp)
vp1, vp2, vp3, pp, roadPlane, focal = tuple_cam
projector = lambda p: getWorldCoordinagesOnRoadPlane(p, focal, roadPlane, pp)
points = map(lambda x,y: np.array([x,y,1]), [5,6], [7,8])
points = map(projector, points)
print(points)

ValueError: incompatible dimensions for cross product
(dimension must be 2 or 3)

In [33]:
a = list(points)

In [63]:
vp1 = [144.737, 34.7794]
vp2 = [12183.582175112755, 615.451021479187]
pp = [960.5, 540.5]
vp1, vp2, vp3, pp, roadPlane, focal = tuple_cam
points = map(lambda p: np.array([p[-2],p[-1],1]), ([5,6],[7,8]))
print(list(points))
# points = map(projector, points)

# print(list(points))

[array([5, 6, 1]), array([7, 8, 1])]
